IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M1\FAULT_M1_10


In [3]:
guasto = 2

VOLO M1 FAULT 10%

In [4]:
rcou_m1_fault10 = pd.read_csv("RCOU.csv")
rcou_m1_fault10 = rcou_m1_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1_fault10 = rcou_m1_fault10[((rcou_m1_fault10['C9'] >= potenza) & (rcou_m1_fault10['C10'] >= potenza) & (rcou_m1_fault10['C11'] >= potenza) & (rcou_m1_fault10['C12'] >= potenza) & (rcou_m1_fault10['C13'] >= potenza) & (rcou_m1_fault10['C14']>= potenza))]

rcou_m1_fault10 = rcou_m1_fault10.reset_index(drop=True)

display(rcou_m1_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,1648547241,1431,1424,1373,1477,1370,1479
1,1648558959,1351,1504,1451,1414,1366,1493
2,1648579931,1467,1420,1429,1459,1448,1441
3,1648600801,1525,1374,1393,1511,1466,1443
4,1648611573,1436,1483,1442,1478,1413,1504
...,...,...,...,...,...,...,...
7959,1749459761,1576,1423,1531,1477,1501,1508
7960,1749470298,1555,1447,1515,1492,1459,1545
7961,1749480739,1516,1484,1534,1465,1440,1556
7962,1749491176,1555,1431,1525,1465,1478,1513


In [5]:
min=rcou_m1_fault10['TimeUS'][0]
max=rcou_m1_fault10['TimeUS'][len(rcou_m1_fault10)-1]
print(max)
print(min)

1749502709
1648547241


In [6]:
xkf1_m1_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m1_fault10 = xkf1_m1_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1_fault10 = xkf1_m1_fault10[((xkf1_m1_fault10['TimeUS'] >= min) & (xkf1_m1_fault10['TimeUS'] <= max))]
xkf1_m1_fault10 = xkf1_m1_fault10.reset_index(drop=True)
print(xkf1_m1_fault10)

          TimeUS  Roll  Pitch    Yaw
0     1648557084 -1.24   1.59  19.96
1     1648567530 -1.34   1.57  19.96
2     1648577440 -1.25   1.57  19.94
3     1648588414 -1.13   1.58  19.94
4     1648598616 -1.03   1.62  19.93
...          ...   ...    ...    ...
9157  1749457351 -1.53   0.06   6.15
9158  1749468272 -1.50   0.07   6.16
9159  1749478311 -1.41   0.00   6.13
9160  1749489170 -1.21  -0.05   6.14
9161  1749500410 -0.92  -0.12   6.20

[9162 rows x 4 columns]


In [7]:
att_m1_fault10 = pd.read_csv("ATT.csv")
att_m1_fault10 = att_m1_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1_fault10 = att_m1_fault10[((att_m1_fault10['TimeUS'] >= min) & (att_m1_fault10['TimeUS'] <= max))]
att_m1_fault10=att_m1_fault10.reset_index(drop=True)
print(att_m1_fault10)

          TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     1648558171    -0.06 -1.24     -0.40   1.59   19.79  19.96
1     1648568640    -0.05 -1.34     -0.41   1.57   19.79  19.96
2     1648579278    -0.04 -1.25     -0.41   1.57   19.79  19.94
3     1648589763    -0.04 -1.13     -0.42   1.58   19.79  19.94
4     1648600108    -0.04 -1.03     -0.43   1.62   19.79  19.93
...          ...      ...   ...       ...    ...     ...    ...
9158  1749459114     0.74 -1.53     -0.32   0.06    8.12   6.15
9159  1749469562     0.75 -1.50     -0.34   0.07    8.12   6.16
9160  1749479836     0.76 -1.41     -0.34   0.00    8.12   6.13
9161  1749490199     0.75 -1.21     -0.33  -0.05    8.12   6.14
9162  1749501809     0.74 -0.92     -0.32  -0.12    8.12   6.20

[9163 rows x 7 columns]


In [8]:
if len(att_m1_fault10)>len(xkf1_m1_fault10):
    to_add=att_m1_fault10[len(xkf1_m1_fault10):]
    att_m1_fault10=att_m1_fault10[:len(xkf1_m1_fault10)]

if len(xkf1_m1_fault10)>len(att_m1_fault10):
    to_add=xkf1_m1_fault10[len(att_m1_fault10):]

for idx,i in enumerate(att_m1_fault10['Roll']):
    if(xkf1_m1_fault10['Roll'][idx] != i):
        att_m1_fault10['Roll'][idx] = ((att_m1_fault10['Roll'][idx] + xkf1_m1_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m1_fault10['Pitch']):
    if(xkf1_m1_fault10['Pitch'][idx] != i):
        att_m1_fault10['Pitch'][idx] = ((att_m1_fault10['Pitch'][idx] + xkf1_m1_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m1_fault10['Yaw']):
    if(xkf1_m1_fault10['Yaw'][idx] != i):
        att_m1_fault10['Yaw'][idx] = ((att_m1_fault10['Yaw'][idx] + xkf1_m1_fault10['Yaw'][idx])/2)

att_m1_fault10=pd.concat([att_m1_fault10,to_add])

print(att_m1_fault10)

C:\Users\mgale\AppData\Local\Temp\ipykernel_23788\4117696472.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault10['Roll'][idx] = ((att_m1_fault10['Roll'][idx] + xkf1_m1_fault10['Roll'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_23788\4117696472.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault10['Pitch'][idx] = ((att_m1_fault10['Pitch'][idx] + xkf1_m1_fault10['Pitch'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_23788\4117696472.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

          TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw
0     1648558171    -0.06 -1.240     -0.40  1.590   19.79  19.960
1     1648568640    -0.05 -1.340     -0.41  1.570   19.79  19.960
2     1648579278    -0.04 -1.250     -0.41  1.570   19.79  19.940
3     1648589763    -0.04 -1.130     -0.42  1.580   19.79  19.940
4     1648600108    -0.04 -1.030     -0.43  1.620   19.79  19.930
...          ...      ...    ...       ...    ...     ...     ...
9158  1749459114     0.74 -1.515     -0.32  0.065    8.12   6.155
9159  1749469562     0.75 -1.455     -0.34  0.035    8.12   6.145
9160  1749479836     0.76 -1.310     -0.34 -0.025    8.12   6.135
9161  1749490199     0.75 -1.065     -0.33 -0.085    8.12   6.170
9162  1749501809     0.74 -0.920     -0.32 -0.120    8.12   6.200

[9163 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m1_fault10 = esc_0_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1_fault10 = esc_0_m1_fault10[((esc_0_m1_fault10['TimeUS'] >= min) & (esc_0_m1_fault10['TimeUS'] <= max))]
esc_0_m1_fault10=esc_0_m1_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m1_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m1_fault10 = esc_1_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1_fault10 = esc_1_m1_fault10[((esc_1_m1_fault10['TimeUS'] >= min) & (esc_1_m1_fault10['TimeUS'] <= max))]
esc_1_m1_fault10=esc_1_m1_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m1_fault10 = esc_2_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1_fault10 = esc_2_m1_fault10[((esc_2_m1_fault10['TimeUS'] >= min) & (esc_2_m1_fault10['TimeUS'] <= max))]
esc_2_m1_fault10=esc_2_m1_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m1_fault10 = esc_3_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1_fault10 = esc_3_m1_fault10[((esc_3_m1_fault10['TimeUS'] >= min) & (esc_3_m1_fault10['TimeUS'] <= max))]
esc_3_m1_fault10=esc_3_m1_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m1_fault10 = esc_4_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1_fault10 = esc_4_m1_fault10[((esc_4_m1_fault10['TimeUS'] >= min) & (esc_4_m1_fault10['TimeUS'] <= max))]
esc_4_m1_fault10=esc_4_m1_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m1_fault10 = esc_5_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1_fault10 = esc_5_m1_fault10[((esc_5_m1_fault10['TimeUS'] >= min) & (esc_5_m1_fault10['TimeUS'] <= max))]
esc_5_m1_fault10=esc_5_m1_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m1_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m1_fault10 = imu_0_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1_fault10 = imu_0_m1_fault10[((imu_0_m1_fault10['TimeUS'] >= min) & (imu_0_m1_fault10['TimeUS'] <= max))]

imu_1_m1_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m1_fault10 = imu_1_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1_fault10 = imu_1_m1_fault10[((imu_1_m1_fault10['TimeUS'] >= min) & (imu_1_m1_fault10['TimeUS'] <= max))]

imu_2_m1_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m1_fault10 = imu_2_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1_fault10 = imu_2_m1_fault10[((imu_2_m1_fault10['TimeUS'] >= min) & (imu_2_m1_fault10['TimeUS'] <= max))]

imu_m1_fault10 = pd.concat((imu_0_m1_fault10, imu_1_m1_fault10, imu_2_m1_fault10))
imu_m1_fault10=imu_m1_fault10.groupby(imu_m1_fault10.TimeUS, as_index=False).mean()

display(imu_m1_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,1648552405,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406
1,1648558068,-0.223346,0.002401,0.015638,0.191140,0.485832,-9.585221
2,1648561764,0.041824,0.100781,0.001137,0.231455,0.761186,-9.726832
3,1648563543,0.220820,0.071107,0.001237,0.379594,0.636334,-9.590402
4,1648566595,0.112270,-0.002903,0.002685,0.414795,0.359073,-9.528167
...,...,...,...,...,...,...,...
34809,1749488276,0.280210,-0.035147,0.002372,0.501059,0.538888,-10.258573
34810,1749490094,0.245654,-0.019537,0.029526,0.445560,0.603047,-10.375337
34811,1749492809,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707
34812,1749499200,0.296264,-0.050650,0.081374,-0.083817,0.647380,-10.938693


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m1_fault10 = pd.merge_ordered(imu_m1_fault10,att_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_0_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_1_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_2_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_3_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_4_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_5_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")
m1_fault10 = pd.merge_ordered(m1_fault10, rcou_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
m1_fault10["TimeUS"] = m1_fault10["TimeUS"] - m1_fault10.iloc[0]["TimeUS"]
m1_fault10["TimeUS"] = m1_fault10["TimeUS"].astype(int)
m1_fault10["Guasto"] = guasto

In [13]:
last_cell = m1_fault10.iloc[-1, m1_fault10.columns.get_loc('TimeUS')]
print(last_cell)

100955468


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

          TimeUS
0              0
1           2857
2           5714
3           8571
4          11428
...          ...
35332  100943524
35333  100946381
35334  100949238
35335  100952095
35336  100954952

[35337 rows x 1 columns]


In [16]:
m1_fault10_final = pd.merge_ordered(m1_fault10,df)
m1_fault10_final=m1_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m1_fault10_final)

          TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0              0  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
1           2857  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
2           5164  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
3           5714  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
4           8571  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
...          ...       ...       ...       ...       ...       ...        ...   
92326  100952095  0.296264 -0.050650  0.081374 -0.083817  0.647380 -10.938693   
92327  100954357  0.355682 -0.062922  0.094976 -0.154238  0.797622 -11.020897   
92328  100954568  0.355682 -0.062922  0.094976 -0.154238  0.797622 -11.020897   
92329  100954952  0.355682 -0.062922  0.094976 -0.154238  0.797622 -11.020897   
92330  100955468  0.355682 -0.062922  0.094976 -0.154238  0.797622 -11.020897   

       DesRoll   Roll  DesP

In [17]:
m1_fault10_final = m1_fault10_final[m1_fault10_final.TimeUS.isin(to_be)]

In [18]:
m1_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
1,2857,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
3,5714,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
4,8571,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
7,11428,-0.223346,0.002401,0.015638,0.191140,0.485832,-9.585221,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92319,100943524,0.245654,-0.019537,0.029526,0.445560,0.603047,-10.375337,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0,2.0
92323,100946381,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
92324,100949238,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
92326,100952095,0.296264,-0.050650,0.081374,-0.083817,0.647380,-10.938693,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0


In [19]:
from datetime import timedelta
m1_fault10_final=m1_fault10_final.reset_index(drop=True)
m1_fault10_final['TimeUS'] = pd.to_datetime(m1_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
1,00:00:00.002857,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
2,00:00:00.005714,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
3,00:00:00.008571,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
4,00:00:00.011428,-0.223346,0.002401,0.015638,0.191140,0.485832,-9.585221,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35332,00:01:40.943524,0.245654,-0.019537,0.029526,0.445560,0.603047,-10.375337,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0,2.0
35333,00:01:40.946381,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
35334,00:01:40.949238,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
35335,00:01:40.952095,0.296264,-0.050650,0.081374,-0.083817,0.647380,-10.938693,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_fault10_final)/350)):
        V1=m1_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_23788\2166352407.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_23788\2166352407.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.032100,0.017081,-0.399284,0.134398,78.0,96.0,3.596543,3.414090,-0.014483,...,5812.008175,3.227550,0.201562,-1.034645,3.258537,120.0,119.0,15.414952,8.888416,2
1,1.0,-0.032346,0.017054,-0.400010,0.134356,102.0,80.0,3.601798,3.359201,-0.014131,...,5704.453240,3.229687,0.202596,-1.035832,3.260813,128.0,127.0,15.447674,8.483140,2
2,2.0,-0.032590,0.017026,-0.400594,0.134314,98.0,79.0,3.785400,3.155365,-0.013642,...,5587.855979,3.231813,0.203615,-1.037158,3.263074,136.0,134.0,15.463306,8.173170,2
3,3.0,-0.032832,0.016999,-0.401113,0.134271,102.0,71.0,3.964024,3.257887,-0.013156,...,5462.279375,3.233927,0.204620,-1.038609,3.265321,130.0,128.0,15.462261,8.871637,2
4,4.0,-0.033265,0.017017,-0.412325,0.134447,102.0,87.0,4.069144,2.898697,-0.013003,...,5327.792013,3.236028,0.205610,-1.040173,3.267554,138.0,135.0,15.444971,9.516132,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95.0,-0.054057,0.016272,-0.525996,0.138411,121.0,125.0,3.995707,3.942622,0.011208,...,7576.684681,3.445381,0.340305,-1.002302,3.494309,156.0,157.0,13.840458,12.661880,2
96,96.0,-0.053752,0.016277,-0.530386,0.138311,115.0,111.0,4.401276,4.117022,0.011439,...,7294.977740,3.447964,0.342524,-0.999154,3.497172,157.0,158.0,14.011608,12.958245,2
97,97.0,-0.054048,0.016280,-0.534119,0.138438,0.0,123.0,3.265261,2.587508,0.011548,...,7003.293938,3.450536,0.344721,-0.996478,3.500021,146.0,147.0,14.160572,13.239533,2
98,98.0,-0.054516,0.016342,-0.541786,0.138844,0.0,123.0,3.240923,2.612877,0.011884,...,6702.845460,3.453096,0.346894,-0.994243,3.502855,159.0,160.0,14.287081,13.505498,2


In [23]:
path_file = path_file.replace(r"M1\FAULT_M1_10", "")
os.chdir(path_file)
df_merged.to_csv('m1_fault10.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
